In [3]:
import numpy as np
import pandas as pd


path = '/opt/ml/input/data/'

train = pd.read_csv(path + 'train_data.csv')
test = pd.read_csv(path + 'test_data.csv')

In [4]:
dataset = pd.read_csv(path + 'train_data.csv', parse_dates=["Timestamp"])
diff = dataset.loc[:, ["userID", "Timestamp"]].groupby("userID").diff().fillna(pd.Timedelta(seconds=0))
diff['Timestamp'] = diff['Timestamp'].apply(pd.Timedelta.total_seconds)
diff['Timestamp'] = diff['Timestamp'].apply(lambda x: 0 if x >600 or x<0 else x)
train['solve_time'] = diff['Timestamp']

In [5]:
print(diff.quantile(0.1))
print(diff.describe())

Timestamp    0.0
Name: 0.1, dtype: float64
          Timestamp
count  2.266586e+06
mean   4.637118e+01
std    7.754732e+01
min    0.000000e+00
25%    3.000000e+00
50%    1.900000e+01
75%    5.200000e+01
max    6.000000e+02


In [6]:
def train_get_category(x):

    if x <= 0:# 0
        return 0
    elif x <= 3:# 25%
        return 1
    elif x <= 19:# 50%
        return 2
    elif x <= 52:# 75%
        return 3
    else:
        return 4

In [7]:
diff['Timestamp'] = diff['Timestamp'].apply(lambda x: train_get_category(x))
diff

,Timestamp
0,0
1,1
2,2
3,2
4,2
...,...
2266581,3
2266582,0
2266583,2
2266584,3


In [8]:
train['category_st'] = diff['Timestamp']
train

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,solve_time,category_st
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0.0,0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,3.0,1
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,8.0,2
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,7.0,2
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,7.0,2
...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,24.0,3
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,0.0,0
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,11.0,2
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,46.0,3


In [9]:
train.to_csv('/opt/ml/input/data/FE/FE_solve_time_train.csv', index=False)

In [10]:
dataset = pd.read_csv(path + 'test_data.csv', parse_dates=["Timestamp"])
diff = dataset.loc[:, ["userID", "Timestamp"]].groupby("userID").diff().fillna(pd.Timedelta(seconds=0))
diff['Timestamp'] = diff['Timestamp'].apply(pd.Timedelta.total_seconds)
diff['Timestamp'] = diff['Timestamp'].apply(lambda x: 0 if x >600 or x<0 else x)
test['solve_time'] = diff['Timestamp']

In [11]:
print(diff.quantile(0.1))
print(diff.describe())

Timestamp    0.0
Name: 0.1, dtype: float64
           Timestamp
count  260114.000000
mean       46.273488
std        77.452764
min         0.000000
25%         3.000000
50%        19.000000
75%        52.000000
max       600.000000


In [12]:
diff['Timestamp'] = diff['Timestamp'].apply(lambda x: train_get_category(x))
diff

,Timestamp
0,0
1,3
2,4
3,2
4,2
...,...
260109,0
260110,2
260111,3
260112,4


In [13]:
test['category_st'] = diff['Timestamp']
test

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,solve_time,category_st
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626,0.0,0
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626,26.0,3
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625,94.0,4
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625,5.0,2
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623,7.0,2
...,...,...,...,...,...,...,...,...
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832,0.0,0
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832,18.0,2
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244,21.0,3
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244,89.0,4


In [14]:
test.to_csv('/opt/ml/input/data/FE/FE_solve_time_test.csv', index=False)